<a href="https://colab.research.google.com/github/nehalmuthu/Abstractive-text-summarization-using-Attention-based-Neural-Network/blob/main/amazonrev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

In [ ]:
!pip install tensorflow==1.1.0

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or 

TensorFlow Version: 1.1.0


In [ ]:
!unzip "/content/drive/My Drive/packages/nlp/easypro/amazon-fine-food-reviews.zip" -d "/content/drive/My Drive/packages/nlp/easypro/amaz"

Archive:  /content/drive/My Drive/packages/nlp/easypro/amazon-fine-food-reviews.zip
replace /content/drive/My Drive/packages/nlp/easypro/amaz/Reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N



In [ ]:
reviews = pd.read_csv("/content/drive/My Drive/packages/nlp/easypro/amaz/Reviews.csv")

In [ ]:
reviews.shape


(568454, 10)

In [ ]:
reviews['Text'][1]

'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".'

In [ ]:
reviews.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Remove NULL data


In [ ]:
reviews.isnull().sum()


Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [ ]:
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator',
                        'Score','Time'], 1)
reviews = reviews.reset_index(drop=True)

In [ ]:
reviews.head()


,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


Replacing contractions with long forms


In [ ]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

Remove unwanted words


In [ ]:
# Clean the summaries and texts
clean_summaries = []
for summary in reviews.Summary:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [ ]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

Clean Review # 1
good quality dog food
bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better

Clean Review # 2
not as advertised
product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo

Clean Review # 3
 delight  says it all
confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story c lewis lion witch wardrobe treat seduces edmund selling brother sisters witch

Clean Review # 4
cough medicine
looking secret ingredient robitussin believe found got addition root beer extract ordered good made cherry soda flavor medicinal

Clean Review # 5
great taffy
great taffy great price wide assortment yummy taffy delivery quick taffy lover deal



Count word frequency


In [ ]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1


In [ ]:
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 132884


Word Embedding


In [ ]:
embeddings_index = {}
with open('/content/drive/My Drive/packages/nlp/easypro/amaz/numberbatch-en-17.02 (1).txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 484557


In [ ]:
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 3044
Percent of words that are missing from vocabulary: 2.29%


In [ ]:
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 132884
Number of words we will use: 65469
Percent of words we will use: 49.27%


In [ ]:
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

65469


In [ ]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [ ]:
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 25679933
Total number of UNKs in headlines: 170450
Percent of words that are UNK: 0.66%


Sort data by length of description

In [ ]:

def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [ ]:

lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
              counts
count  568411.000000
mean        4.181624
std         2.657872
min         0.000000
25%         2.000000
50%         4.000000
75%         5.000000
max        48.000000

Texts:
              counts
count  568411.000000
mean       41.996835
std        42.520873
min         1.000000
25%        18.000000
50%        29.000000
75%        50.000000
max      2085.000000


In [ ]:
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

84.0
115.0
207.0


In [ ]:
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

8.0
9.0
13.0


In [ ]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [ ]:

sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

429209
429209


Model Building



In [ ]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [ ]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [ ]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [ ]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state[0],
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32)) 
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)

    return training_logits, inference_logits

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [ ]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:

def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [ ]:

# Set the Hyperparameters
epochs = 10
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [ ]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Graph is built.


In [ ]:
# Subset the data for training
start = 200000

end = start + 50000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 25
The longest text length: 31


In [ ]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/10 Batch   20/781 - Loss:  4.542, Seconds: 233.00
Epoch   1/10 Batch   40/781 - Loss:  2.840, Seconds: 181.33
Epoch   1/10 Batch   60/781 - Loss:  2.814, Seconds: 200.09
Epoch   1/10 Batch   80/781 - Loss:  2.805, Seconds: 182.39
Epoch   1/10 Batch  100/781 - Loss:  2.839, Seconds: 165.16
Epoch   1/10 Batch  120/781 - Loss:  2.778, Seconds: 198.36
Epoch   1/10 Batch  140/781 - Loss:  2.708, Seconds: 230.77
Epoch   1/10 Batch  160/781 - Loss:  2.676, Seconds: 166.04
Epoch   1/10 Batch  180/781 - Loss:  2.550, Seconds: 182.29
Epoch   1/10 Batch  200/781 - Loss:  2.620, Seconds: 216.55
Epoch   1/10 Batch  220/781 - Loss:  2.671, Seconds: 215.42
Epoch   1/10 Batch  240/781 - Loss:  2.654, Seconds: 216.33
Average loss for this update: 2.851
New Record!
Epoch   1/10 Batch  260/781 - Loss:  2.537, Seconds: 231.70
Epoch   1/10 Batch  280/781 - Loss:  2.445, Seconds: 220.03
Epoch   1/10 Batch  300/781 - Loss:  2.339, Seconds: 200.95
Epoch   1/10 Batch  320/781 - Loss:  2.489, Seconds:

In [ ]:
Epoch   1/100 Batch   20/781 - Loss:  4.470, Seconds: 156.00
Epoch   1/100 Batch   40/781 - Loss:  2.863, Seconds: 105.20
Epoch   1/100 Batch   60/781 - Loss:  2.652, Seconds: 151.58
Epoch   1/100 Batch   80/781 - Loss:  2.736, Seconds: 117.19
Epoch   1/100 Batch  100/781 - Loss:  2.686, Seconds: 118.42
Epoch   1/100 Batch  120/781 - Loss:  2.423, Seconds: 140.21
Epoch   1/100 Batch  140/781 - Loss:  2.696, Seconds: 152.89
Epoch   1/100 Batch  160/781 - Loss:  2.606, Seconds: 128.19
Epoch   1/100 Batch  180/781 - Loss:  2.525, Seconds: 151.52
Epoch   1/100 Batch  200/781 - Loss:  2.597, Seconds: 140.84
Epoch   1/100 Batch  220/781 - Loss:  2.515, Seconds: 130.87
Epoch   1/100 Batch  240/781 - Loss:  2.402, Seconds: 131.02
Average loss for this update: 2.734
New Record!
Epoch   1/100 Batch  260/781 - Loss:  2.382, Seconds: 106.18
Epoch   1/100 Batch  280/781 - Loss:  2.354, Seconds: 124.90
Epoch   1/100 Batch  300/781 - Loss:  2.306, Seconds: 148.73
Epoch   1/100 Batch  320/781 - Loss:  2.637, Seconds: 142.09
Epoch   1/100 Batch  340/781 - Loss:  2.680, Seconds: 140.91
Epoch   1/100 Batch  360/781 - Loss:  2.559, Seconds: 96.81
Epoch   1/100 Batch  380/781 - Loss:  2.448, Seconds: 130.00
Epoch   1/100 Batch  400/781 - Loss:  2.615, Seconds: 108.58
Epoch   1/100 Batch  420/781 - Loss:  2.193, Seconds: 124.55
Epoch   1/100 Batch  440/781 - Loss:  2.315, Seconds: 131.78
Epoch   1/100 Batch  460/781 - Loss:  2.276, Seconds: 131.26
Epoch   1/100 Batch  480/781 - Loss:  2.391, Seconds: 88.43
Epoch   1/100 Batch  500/781 - Loss:  2.455, Seconds: 120.87
Average loss for this update: 2.436
New Record!
Epoch   1/100 Batch  520/781 - Loss:  2.459, Seconds: 94.69
Epoch   1/100 Batch  540/781 - Loss:  2.412, Seconds: 108.36
Epoch   1/100 Batch  560/781 - Loss:  2.320, Seconds: 128.48
Epoch   1/100 Batch  580/781 - Loss:  2.195, Seconds: 121.47
Epoch   1/100 Batch  600/781 - Loss:  2.353, Seconds: 120.64
Epoch   1/100 Batch  620/781 - Loss:  2.079, Seconds: 153.50
Epoch   1/100 Batch  640/781 - Loss:  2.254, Seconds: 154.52
Epoch   1/100 Batch  660/781 - Loss:  2.570, Seconds: 124.24
Epoch   1/100 Batch  680/781 - Loss:  2.352, Seconds: 154.14
Epoch   1/100 Batch  700/781 - Loss:  2.170, Seconds: 121.13
Epoch   1/100 Batch  720/781 - Loss:  2.159, Seconds: 121.35
Epoch   1/100 Batch  740/781 - Loss:  2.201, Seconds: 121.36
Epoch   1/100 Batch  760/781 - Loss:  2.037, Seconds: 122.65
Average loss for this update: 2.253
New Record!
Epoch   1/100 Batch  780/781 - Loss:  2.088, Seconds: 136.48
Epoch   2/100 Batch   20/781 - Loss:  2.298, Seconds: 152.02
Epoch   2/100 Batch   40/781 - Loss:  2.171, Seconds: 108.29
Epoch   2/100 Batch   60/781 - Loss:  2.015, Seconds: 152.19
Epoch   2/100 Batch   80/781 - Loss:  2.052, Seconds: 117.14
Epoch   2/100 Batch  100/781 - Loss:  2.069, Seconds: 119.78
Epoch   2/100 Batch  120/781 - Loss:  1.831, Seconds: 145.87
Epoch   2/100 Batch  140/781 - Loss:  2.136, Seconds: 151.15
Epoch   2/100 Batch  160/781 - Loss:  2.082, Seconds: 132.47
Epoch   2/100 Batch  180/781 - Loss:  2.019, Seconds: 153.08
Epoch   2/100 Batch  200/781 - Loss:  2.052, Seconds: 142.47
Epoch   2/100 Batch  220/781 - Loss:  1.969, Seconds: 130.85
Epoch   2/100 Batch  240/781 - Loss:  1.819, Seconds: 129.27
Average loss for this update: 2.028
New Record!
Epoch   2/100 Batch  260/781 - Loss:  1.869, Seconds: 103.74
Epoch   2/100 Batch  280/781 - Loss:  1.829, Seconds: 130.26
Epoch   2/100 Batch  300/781 - Loss:  1.803, Seconds: 151.96
Epoch   2/100 Batch  320/781 - Loss:  2.139, Seconds: 141.95
Epoch   2/100 Batch  340/781 - Loss:  2.189, Seconds: 145.17
Epoch   2/100 Batch  360/781 - Loss:  2.047, Seconds: 98.12
Epoch   2/100 Batch  380/781 - Loss:  1.955, Seconds: 130.35
Epoch   2/100 Batch  400/781 - Loss:  2.080, Seconds: 108.95
Epoch   2/100 Batch  420/781 - Loss:  1.717, Seconds: 120.29
Epoch   2/100 Batch  440/781 - Loss:  1.842, Seconds: 130.07
Epoch   2/100 Batch  460/781 - Loss:  1.798, Seconds: 132.52
Epoch   2/100 Batch  480/781 - Loss:  1.929, Seconds: 87.39
Epoch   2/100 Batch  500/781 - Loss:  2.050, Seconds: 122.04
Average loss for this update: 1.953
New Record!
Epoch   2/100 Batch  520/781 - Loss:  2.037, Seconds: 94.28
Epoch   2/100 Batch  540/781 - Loss:  1.972, Seconds: 109.17
Epoch   2/100 Batch  560/781 - Loss:  1.923, Seconds: 131.12
Epoch   2/100 Batch  580/781 - Loss:  1.728, Seconds: 120.70
Epoch   2/100 Batch  600/781 - Loss:  1.894, Seconds: 120.67
Epoch   2/100 Batch  620/781 - Loss:  1.708, Seconds: 154.05
Epoch   2/100 Batch  640/781 - Loss:  1.859, Seconds: 153.93
Epoch   2/100 Batch  660/781 - Loss:  2.167, Seconds: 121.69
Epoch   2/100 Batch  680/781 - Loss:  1.982, Seconds: 154.94
Epoch   2/100 Batch  700/781 - Loss:  1.804, Seconds: 122.34
Epoch   2/100 Batch  720/781 - Loss:  1.800, Seconds: 120.82
Epoch   2/100 Batch  740/781 - Loss:  1.778, Seconds: 121.45
Epoch   2/100 Batch  760/781 - Loss:  1.712, Seconds: 121.23
Average loss for this update: 1.857
New Record!
Epoch   2/100 Batch  780/781 - Loss:  1.737, Seconds: 136.31
Epoch   3/100 Batch   20/781 - Loss:  2.015, Seconds: 150.41
Epoch   3/100 Batch   40/781 - Loss:  1.906, Seconds: 108.50
Epoch   3/100 Batch   60/781 - Loss:  1.721, Seconds: 155.11
Epoch   3/100 Batch   80/781 - Loss:  1.728, Seconds: 117.11
Epoch   3/100 Batch  100/781 - Loss:  1.776, Seconds: 121.61
Epoch   3/100 Batch  120/781 - Loss:  1.569, Seconds: 141.59
Epoch   3/100 Batch  140/781 - Loss:  1.845, Seconds: 152.36
Epoch   3/100 Batch  160/781 - Loss:  1.811, Seconds: 129.96
Epoch   3/100 Batch  180/781 - Loss:  1.749, Seconds: 153.36
Epoch   3/100 Batch  200/781 - Loss:  1.770, Seconds: 139.69
Epoch   3/100 Batch  220/781 - Loss:  1.698, Seconds: 133.17
Epoch   3/100 Batch  240/781 - Loss:  1.535, Seconds: 130.97
Average loss for this update: 1.746
New Record!
Epoch   3/100 Batch  260/781 - Loss:  1.594, Seconds: 103.10
Epoch   3/100 Batch  280/781 - Loss:  1.549, Seconds: 130.66
Epoch   3/100 Batch  300/781 - Loss:  1.524, Seconds: 151.88
Epoch   3/100 Batch  320/781 - Loss:  1.861, Seconds: 142.28
Epoch   3/100 Batch  340/781 - Loss:  1.893, Seconds: 139.69
Epoch   3/100 Batch  360/781 - Loss:  1.769, Seconds: 98.39
Epoch   3/100 Batch  380/781 - Loss:  1.689, Seconds: 131.38
Epoch   3/100 Batch  400/781 - Loss:  1.803, Seconds: 109.51
Epoch   3/100 Batch  420/781 - Loss:  1.455, Seconds: 122.05
Epoch   3/100 Batch  440/781 - Loss:  1.588, Seconds: 132.11
Epoch   3/100 Batch  460/781 - Loss:  1.540, Seconds: 130.48
Epoch   3/100 Batch  480/781 - Loss:  1.687, Seconds: 88.24
Epoch   3/100 Batch  500/781 - Loss:  1.818, Seconds: 121.00
Average loss for this update: 1.688
New Record!
Epoch   3/100 Batch  520/781 - Loss:  1.787, Seconds: 94.69
Epoch   3/100 Batch  540/781 - Loss:  1.689, Seconds: 109.86
Epoch   3/100 Batch  560/781 - Loss:  1.678, Seconds: 131.70
Epoch   3/100 Batch  580/781 - Loss:  1.488, Seconds: 120.36
Epoch   3/100 Batch  600/781 - Loss:  1.637, Seconds: 122.60
Epoch   3/100 Batch  620/781 - Loss:  1.493, Seconds: 153.82
Epoch   3/100 Batch  640/781 - Loss:  1.627, Seconds: 158.39
Epoch   3/100 Batch  660/781 - Loss:  1.910, Seconds: 121.86
Epoch   3/100 Batch  680/781 - Loss:  1.755, Seconds: 153.93
Epoch   3/100 Batch  700/781 - Loss:  1.590, Seconds: 121.65
Epoch   3/100 Batch  720/781 - Loss:  1.585, Seconds: 122.38
Epoch   3/100 Batch  740/781 - Loss:  1.540, Seconds: 122.20
Epoch   3/100 Batch  760/781 - Loss:  1.515, Seconds: 122.62
Average loss for this update: 1.623
New Record!
Epoch   3/100 Batch  780/781 - Loss:  1.515, Seconds: 135.59
Epoch   4/100 Batch   20/781 - Loss:  1.793, Seconds: 152.96
Epoch   4/100 Batch   40/781 - Loss:  1.713, Seconds: 107.82
Epoch   4/100 Batch   60/781 - Loss:  1.527, Seconds: 153.71
Epoch   4/100 Batch   80/781 - Loss:  1.513, Seconds: 118.78
Epoch   4/100 Batch  100/781 - Loss:  1.577, Seconds: 121.06
Epoch   4/100 Batch  120/781 - Loss:  1.397, Seconds: 143.41
Epoch   4/100 Batch  140/781 - Loss:  1.635, Seconds: 152.24
Epoch   4/100 Batch  160/781 - Loss:  1.616, Seconds: 131.96
Epoch   4/100 Batch  180/781 - Loss:  1.570, Seconds: 152.69
Epoch   4/100 Batch  200/781 - Loss:  1.580, Seconds: 141.25
Epoch   4/100 Batch  220/781 - Loss:  1.510, Seconds: 131.03
Epoch   4/100 Batch  240/781 - Loss:  1.337, Seconds: 129.62
Average loss for this update: 1.551
New Record!
Epoch   4/100 Batch  260/781 - Loss:  1.416, Seconds: 104.76
Epoch   4/100 Batch  280/781 - Loss:  1.371, Seconds: 130.99
Epoch   4/100 Batch  300/781 - Loss:  1.353, Seconds: 150.54
Epoch   4/100 Batch  320/781 - Loss:  1.648, Seconds: 142.99
Epoch   4/100 Batch  340/781 - Loss:  1.692, Seconds: 142.90
Epoch   4/100 Batch  360/781 - Loss:  1.584, Seconds: 98.49
Epoch   4/100 Batch  380/781 - Loss:  1.505, Seconds: 136.00
Epoch   4/100 Batch  400/781 - Loss:  1.604, Seconds: 110.08
Epoch   4/100 Batch  420/781 - Loss:  1.286, Seconds: 121.17
Epoch   4/100 Batch  440/781 - Loss:  1.411, Seconds: 130.35
Epoch   4/100 Batch  460/781 - Loss:  1.384, Seconds: 131.63
Epoch   4/100 Batch  480/781 - Loss:  1.495, Seconds: 86.78
Epoch   4/100 Batch  500/781 - Loss:  1.641, Seconds: 120.57
Average loss for this update: 1.505
New Record!
Epoch   4/100 Batch  520/781 - Loss:  1.616, Seconds: 94.21
Epoch   4/100 Batch  540/781 - Loss:  1.508, Seconds: 109.50
Epoch   4/100 Batch  560/781 - Loss:  1.489, Seconds: 131.04
Epoch   4/100 Batch  580/781 - Loss:  1.307, Seconds: 122.48
Epoch   4/100 Batch  600/781 - Loss:  1.456, Seconds: 121.87
Epoch   4/100 Batch  620/781 - Loss:  1.348, Seconds: 153.92
Epoch   4/100 Batch  640/781 - Loss:  1.453, Seconds: 156.55
Epoch   4/100 Batch  660/781 - Loss:  1.715, Seconds: 122.58
Epoch   4/100 Batch  680/781 - Loss:  1.590, Seconds: 154.58
Epoch   4/100 Batch  700/781 - Loss:  1.429, Seconds: 123.25
Epoch   4/100 Batch  720/781 - Loss:  1.440, Seconds: 120.86
Epoch   4/100 Batch  740/781 - Loss:  1.379, Seconds: 119.59
Epoch   4/100 Batch  760/781 - Loss:  1.374, Seconds: 123.34
Average loss for this update: 1.456
New Record!
Epoch   4/100 Batch  780/781 - Loss:  1.364, Seconds: 136.66
Epoch   5/100 Batch   20/781 - Loss:  1.621, Seconds: 153.30
Epoch   5/100 Batch   40/781 - Loss:  1.567, Seconds: 106.96
Epoch   5/100 Batch   60/781 - Loss:  1.385, Seconds: 151.55
Epoch   5/100 Batch   80/781 - Loss:  1.366, Seconds: 117.78
Epoch   5/100 Batch  100/781 - Loss:  1.419, Seconds: 120.49
Epoch   5/100 Batch  120/781 - Loss:  1.270, Seconds: 140.66
Epoch   5/100 Batch  140/781 - Loss:  1.478, Seconds: 153.51
Epoch   5/100 Batch  160/781 - Loss:  1.474, Seconds: 131.34
Epoch   5/100 Batch  180/781 - Loss:  1.425, Seconds: 152.85
Epoch   5/100 Batch  200/781 - Loss:  1.437, Seconds: 143.59
Epoch   5/100 Batch  220/781 - Loss:  1.374, Seconds: 133.55
Epoch   5/100 Batch  240/781 - Loss:  1.216, Seconds: 130.37
Average loss for this update: 1.406
New Record!
Epoch   5/100 Batch  260/781 - Loss:  1.271, Seconds: 103.74
Epoch   5/100 Batch  280/781 - Loss:  1.266, Seconds: 130.39
Epoch   5/100 Batch  300/781 - Loss:  1.227, Seconds: 155.28
Epoch   5/100 Batch  320/781 - Loss:  1.486, Seconds: 143.04
Epoch   5/100 Batch  340/781 - Loss:  1.552, Seconds: 141.86
Epoch   5/100 Batch  360/781 - Loss:  1.423, Seconds: 99.56
Epoch   5/100 Batch  380/781 - Loss:  1.365, Seconds: 131.81
Epoch   5/100 Batch  400/781 - Loss:  1.456, Seconds: 113.05
Epoch   5/100 Batch  420/781 - Loss:  1.178, Seconds: 119.84
Epoch   5/100 Batch  440/781 - Loss:  1.290, Seconds: 131.00
Epoch   5/100 Batch  460/781 - Loss:  1.240, Seconds: 133.82
Epoch   5/100 Batch  480/781 - Loss:  1.355, Seconds: 89.87
Epoch   5/100 Batch  500/781 - Loss:  1.496, Seconds: 122.93
Average loss for this update: 1.368
New Record!
Epoch   5/100 Batch  520/781 - Loss:  1.464, Seconds: 96.14
Epoch   5/100 Batch  540/781 - Loss:  1.356, Seconds: 109.62
Epoch   5/100 Batch  560/781 - Loss:  1.362, Seconds: 133.67
Epoch   5/100 Batch  580/781 - Loss:  1.194, Seconds: 121.81
Epoch   5/100 Batch  600/781 - Loss:  1.342, Seconds: 121.25
Epoch   5/100 Batch  620/781 - Loss:  1.239, Seconds: 156.98
Epoch   5/100 Batch  640/781 - Loss:  1.323, Seconds: 156.76
Epoch   5/100 Batch  660/781 - Loss:  1.556, Seconds: 122.91
Epoch   5/100 Batch  680/781 - Loss:  1.453, Seconds: 158.00
Epoch   5/100 Batch  700/781 - Loss:  1.308, Seconds: 123.65
Epoch   5/100 Batch  720/781 - Loss:  1.317, Seconds: 123.67
Epoch   5/100 Batch  740/781 - Loss:  1.267, Seconds: 125.47
Epoch   5/100 Batch  760/781 - Loss:  1.275, Seconds: 122.67
Average loss for this update: 1.331
New Record!
Epoch   5/100 Batch  780/781 - Loss:  1.254, Seconds: 136.56
Epoch   6/100 Batch   20/781 - Loss:  1.486, Seconds: 152.52
Epoch   6/100 Batch   40/781 - Loss:  1.449, Seconds: 111.01
Epoch   6/100 Batch   60/781 - Loss:  1.277, Seconds: 152.59
Epoch   6/100 Batch   80/781 - Loss:  1.241, Seconds: 120.53
Epoch   6/100 Batch  100/781 - Loss:  1.297, Seconds: 120.91
Epoch   6/100 Batch  120/781 - Loss:  1.159, Seconds: 144.18
Epoch   6/100 Batch  140/781 - Loss:  1.360, Seconds: 158.39
Epoch   6/100 Batch  160/781 - Loss:  1.336, Seconds: 132.86
Epoch   6/100 Batch  180/781 - Loss:  1.293, Seconds: 155.74
Epoch   6/100 Batch  200/781 - Loss:  1.321, Seconds: 145.67
Epoch   6/100 Batch  220/781 - Loss:  1.267, Seconds: 135.30
Epoch   6/100 Batch  240/781 - Loss:  1.100, Seconds: 133.28
Average loss for this update: 1.288
New Record!
Epoch   6/100 Batch  260/781 - Loss:  1.168, Seconds: 103.96
Epoch   6/100 Batch  280/781 - Loss:  1.156, Seconds: 131.70
Epoch   6/100 Batch  300/781 - Loss:  1.133, Seconds: 155.86
Epoch   6/100 Batch  320/781 - Loss:  1.361, Seconds: 145.41
Epoch   6/100 Batch  340/781 - Loss:  1.412, Seconds: 145.41
Epoch   6/100 Batch  360/781 - Loss:  1.329, Seconds: 100.96
Epoch   6/100 Batch  380/781 - Loss:  1.266, Seconds: 134.49
Epoch   6/100 Batch  400/781 - Loss:  1.314, Seconds: 110.95
Epoch   6/100 Batch  420/781 - Loss:  1.093, Seconds: 126.54
Epoch   6/100 Batch  440/781 - Loss:  1.206, Seconds: 135.67
Epoch   6/100 Batch  460/781 - Loss:  1.151, Seconds: 137.55
Epoch   6/100 Batch  480/781 - Loss:  1.241, Seconds: 90.75
Epoch   6/100 Batch  500/781 - Loss:  1.377, Seconds: 125.70
Average loss for this update: 1.26
New Record!
Epoch   6/100 Batch  520/781 - Loss:  1.361, Seconds: 94.51
Epoch   6/100 Batch  540/781 - Loss:  1.250, Seconds: 113.06
Epoch   6/100 Batch  560/781 - Loss:  1.268, Seconds: 134.60
Epoch   6/100 Batch  580/781 - Loss:  1.088, Seconds: 123.48
Epoch   6/100 Batch  600/781 - Loss:  1.224, Seconds: 123.78
Epoch   6/100 Batch  620/781 - Loss:  1.153, Seconds: 156.48
Epoch   6/100 Batch  640/781 - Loss:  1.231, Seconds: 157.48
Epoch   6/100 Batch  660/781 - Loss:  1.447, Seconds: 123.59
Epoch   6/100 Batch  680/781 - Loss:  1.363, Seconds: 157.39
Epoch   6/100 Batch  700/781 - Loss:  1.227, Seconds: 128.72
Epoch   6/100 Batch  720/781 - Loss:  1.238, Seconds: 126.35
Epoch   6/100 Batch  740/781 - Loss:  1.174, Seconds: 125.93
Epoch   6/100 Batch  760/781 - Loss:  1.191, Seconds: 125.24
Average loss for this update: 1.239
New Record!
Epoch   6/100 Batch  780/781 - Loss:  1.206, Seconds: 139.27
Epoch   7/100 Batch   20/781 - Loss:  1.416, Seconds: 155.27
Epoch   7/100 Batch   40/781 - Loss:  1.381, Seconds: 110.29
Epoch   7/100 Batch   60/781 - Loss:  1.193, Seconds: 156.02
Epoch   7/100 Batch   80/781 - Loss:  1.177, Seconds: 122.76
Epoch   7/100 Batch  100/781 - Loss:  1.233, Seconds: 122.76
Epoch   7/100 Batch  120/781 - Loss:  1.110, Seconds: 144.25
Epoch   7/100 Batch  140/781 - Loss:  1.274, Seconds: 156.67
Epoch   7/100 Batch  160/781 - Loss:  1.251, Seconds: 138.95
Epoch   7/100 Batch  180/781 - Loss:  1.230, Seconds: 159.33
Epoch   7/100 Batch  200/781 - Loss:  1.244, Seconds: 147.36
Epoch   7/100 Batch  220/781 - Loss:  1.189, Seconds: 136.09
Epoch   7/100 Batch  240/781 - Loss:  1.037, Seconds: 136.38
Average loss for this update: 1.217
New Record!
Epoch   7/100 Batch  260/781 - Loss:  1.106, Seconds: 103.50
Epoch   7/100 Batch  280/781 - Loss:  1.092, Seconds: 135.91
Epoch   7/100 Batch  300/781 - Loss:  1.064, Seconds: 156.83
Epoch   7/100 Batch  320/781 - Loss:  1.267, Seconds: 146.38
Epoch   7/100 Batch  340/781 - Loss:  1.321, Seconds: 145.75
Epoch   7/100 Batch  360/781 - Loss:  1.238, Seconds: 101.32
Epoch   7/100 Batch  380/781 - Loss:  1.187, Seconds: 133.62
Epoch   7/100 Batch  400/781 - Loss:  1.232, Seconds: 114.49
Epoch   7/100 Batch  420/781 - Loss:  1.007, Seconds: 126.89
Epoch   7/100 Batch  440/781 - Loss:  1.119, Seconds: 140.93
Epoch   7/100 Batch  460/781 - Loss:  1.067, Seconds: 139.15
Epoch   7/100 Batch  480/781 - Loss:  1.148, Seconds: 91.13
Epoch   7/100 Batch  500/781 - Loss:  1.280, Seconds: 126.78
Average loss for this update: 1.175
New Record!
Epoch   7/100 Batch  520/781 - Loss:  1.278, Seconds: 126.51
Epoch   7/100 Batch  540/781 - Loss:  1.166, Seconds: 116.39
Epoch   7/100 Batch  560/781 - Loss:  1.180, Seconds: 176.67
Epoch   7/100 Batch  580/781 - Loss:  1.019, Seconds: 123.86
Epoch   7/100 Batch  600/781 - Loss:  1.155, Seconds: 160.57
Epoch   7/100 Batch  620/781 - Loss:  1.081, Seconds: 229.01
Epoch   7/100 Batch  640/781 - Loss:  1.144, Seconds: 213.84
Epoch   7/100 Batch  660/781 - Loss:  1.352, Seconds: 215.01
Epoch   7/100 Batch  680/781 - Loss:  1.281, Seconds: 197.15
Epoch   7/100 Batch  700/781 - Loss:  1.159, Seconds: 195.18
Epoch   7/100 Batch  720/781 - Loss:  1.155, Seconds: 210.13
Epoch   7/100 Batch  740/781 - Loss:  1.104, Seconds: 187.91
Epoch   7/100 Batch  760/781 - Loss:  1.114, Seconds: 176.62
Average loss for this update: 1.159



In [ ]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [ ]:
# Create your own review or use one from the dataset
#input_sentence = "I have never eaten an apple before, but this red one was nice. \
                  #I think that I will try a green apple next time."
#text = text_to_seq(input_sentence)
random = np.random.randint(0,len(clean_texts))
input_sentence = clean_texts[random]
text = text_to_seq(clean_texts[random])

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Text:', input_sentence)

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

In [ ]:
INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Text: love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets

Text
  Word Ids:    [70595, 18808, 668, 45565, 51927, 51759, 32488, 13510, 32036, 59599, 11693, 444, 23335, 32036, 59599, 51927, 67316, 726, 24842, 50494, 48492, 1062, 44749, 38443, 42344, 67973, 14168, 7759, 5347, 29528, 58763, 18927, 17701, 20232, 47328]
  Input Words: love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets

Summary
  Word Ids:       [70595, 28738]
  Response Words: love it



In [ ]:
Examples of reviews and summaries:
Review(1): The coffee tasted great and was at such a good price! I highly recommend this to everyone!
Summary(1): great coffee
Review(2): This is the worst cheese that I have ever bought! I will never buy it again and I hope you won't either!
Summary(2): omg gross gross
Review(3): love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets
Summary(3): love it